# Fooling LIME and SHAP
## Hyperparameter Sensitivity (10)

Analyze the impact of the hyperparameters of LIME and SHAP (e.g., hyperparameters of the local model and of the pertubation algorithms).

In [1]:
import os

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from fooling_LIME_SHAP_Interface.adversarial_model_toolbox import AdversarialModelToolbox
from fooling_LIME_SHAP_Interface.perturbator import Perturbator
from fooling_LIME_SHAP_Interface.util.explainer_type import ExplainerType
from fooling_LIME_SHAP_Interface.util.ml_type import MLType
from fooling_LIME_SHAP_Interface.util.pertubation_method import PerturbationMethod

First we define all the neccessary variables.

In [2]:
TRAIN_TEST_SPLIT = 0.2
SEED = 666

dataset_name = 'HousingData'
input_ids = list(range(13))
categorical_input_ids = []
output_id = 13
biased_id = 11  # B

categorical_input_ids = [input_ids.index(cat_id) for cat_id in categorical_input_ids]
biased_id = input_ids.index(biased_id)

Loading the dataset and doing some preprocessing. There are two datasets. One Dataset is with the biased feature and the other dataset is unbiased because all rows from the biased feature are set to 0.

In [3]:
# Load dataset
house_df = pd.read_csv(os.path.join("datasets", dataset_name + ".csv"))
house_df = house_df.fillna(0)

# Split input and output
y_df = house_df.iloc[:, output_id]
x_df = house_df.iloc[:, input_ids]

scaler = MinMaxScaler()
scaler.fit(x_df.to_numpy())
x = scaler.transform(x_df.to_numpy())

# Create train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y_df.to_numpy(), test_size=TRAIN_TEST_SPLIT, random_state=SEED)

# Create training data without biased column
ux_train = x_train.copy()
ux_train[:, biased_id] = 0
ux_test = x_test.copy()
ux_test[:, biased_id] = 0

Train the biased and unbiased ML model

In [4]:
biased_ml = RandomForestRegressor(random_state=SEED)
biased_ml.fit(x_train, y_train)
print("Accuracy of biased model: {0:3.2}".format(biased_ml.score(x_test, y_test)))

Accuracy of biased model: 0.82


In [5]:
unbiased_ml = RandomForestRegressor(random_state=SEED)
unbiased_ml.fit(ux_test, y_test)
print("Accuracy of unbiased model: {0:3.2}".format(unbiased_ml.score(ux_test, y_test)))

Accuracy of unbiased model: 0.96


for estimators in [1, 5, 10, 20]:
    print("============================== n_kmeans: {} ==============================".format(estimators))
    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.SHAP,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=estimators, perturbator=Perturbator(PerturbationMethod.SUBSTITUTIONS, perturbation_multiplier=10, n_kmeans=10, n_samples=2e4, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)## Hyperparameter sensitivity local model

In [6]:
for hp in [1, 10, 100, 200]:
    print("============================== rf_estimators: {} ==============================".format(hp))
    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=hp, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=30, perturbation_std=0.3, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== rf_estimators: 1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 3.9245022283364555, 'LSTAT': -2.930593439596396, 'DIS': -1.8667575787751114, 'CRIM': -0.5755443443865172, 'TAX': -0.5501921125595892, 'PTRATIO': -0.5101799545223372, 'NOX': -0.45925639555863423, 'AGE': -0.031155411678851865, 'B': 0.05869295737572275, 'CHAS': 0.014586099595466745, 'ZN': 0.03438975601120789, 'RAD': 0.020245962944362896, 'INDUS': 0.01703075450561726}
Adversarial Lime explanation:
{'LSTAT': -4.620809295155651, 'RM': 2.107724842231777, 'DIS': -1.1443293479857988, 'CRIM': 0.702925264076124, 'AGE': -0.21594084145606635, 'PTRATIO': -0.26072617190938885, 'NOX': -0.1153618531182298, 'TAX': -0.0651852367594385, 'RAD': 0.00024684444994439107, 'CHAS': 0.0028184915642736038, 'INDUS': -0.0879568865179662, 'B': -0.00044577468381569305, 'ZN': 0.0007857395341092149}
Prediction fidelity between original and adversarial model: 1.0
=================

## Hyperparameter sensitivity pertubation multiplier

In [7]:
for hp in [1, 10, 20, 30]:
    print("============================== perturbation_multiplier: {} ==============================".format(hp))
    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=100, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=hp, perturbation_std=0.3, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== perturbation_multiplier: 1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 3.9245022283364555, 'LSTAT': -2.930593439596396, 'DIS': -1.8667575787751114, 'CRIM': -0.5755443443865172, 'TAX': -0.5501921125595892, 'PTRATIO': -0.5101799545223372, 'NOX': -0.45925639555863423, 'AGE': -0.031155411678851865, 'B': 0.05869295737572275, 'CHAS': 0.014586099595466745, 'ZN': 0.03438975601120789, 'RAD': 0.020245962944362896, 'INDUS': 0.01703075450561726}
Adversarial Lime explanation:
{'LSTAT': -4.638569632075987, 'RM': 2.0895355183958046, 'DIS': -1.1393672445499108, 'CRIM': 0.7058473349208569, 'AGE': -0.21868471642611037, 'PTRATIO': -0.26096737224508204, 'NOX': -0.11430644363189214, 'TAX': -0.06541496461695044, 'CHAS': 0.004001222661659427, 'RAD': -0.0019163011827034414, 'INDUS': -0.08987151642199523, 'B': -0.001046207358877807, 'ZN': 0.002687561733834433}
Prediction fidelity between original and adversarial model: 1.0
======

## Hyperparameter sensitivity pertubation std

In [8]:
for hp in [0.1, 0.25, 0.5, 0.75, 1.0]:
    print("============================== perturbation_std: {} ==============================".format(hp))

    adv = AdversarialModelToolbox(biased_model=biased_ml, 
                                  x_train=x_train, 
                                  y_train=y_train, 
                                  x_test=x_test, 
                                  y_test=y_test,
                                  input_feature_names=house_df.columns[input_ids].tolist(),
                                  categorical_feature_indices=categorical_input_ids,
                                  unbiased_model=unbiased_ml,
                                  biased_id=biased_id, 
                                  fool_explainer_type=ExplainerType.LIME,
                                  ml_type=MLType.REGRESSION, 
                                  seed=SEED)
    adv.train(rf_estimators=100, perturbator=Perturbator(PerturbationMethod.LIME, perturbation_multiplier=30, perturbation_std=hp, seed=SEED))
    adv.get_explanations(explanation_sample_number=-1)

============================== perturbation_std: 0.1 ==============================
Calculating Lime explanations
Original Lime explanation:
{'RM': 3.9245022283364555, 'LSTAT': -2.930593439596396, 'DIS': -1.8667575787751114, 'CRIM': -0.5755443443865172, 'TAX': -0.5501921125595892, 'PTRATIO': -0.5101799545223372, 'NOX': -0.45925639555863423, 'AGE': -0.031155411678851865, 'B': 0.05869295737572275, 'CHAS': 0.014586099595466745, 'ZN': 0.03438975601120789, 'RAD': 0.020245962944362896, 'INDUS': 0.01703075450561726}
Adversarial Lime explanation:
{'LSTAT': -4.657087939530677, 'RM': 2.061618253444168, 'DIS': -1.1395513470737793, 'CRIM': 0.708440871859505, 'AGE': -0.21768078969010057, 'PTRATIO': -0.25657865002752533, 'NOX': -0.1125827246748999, 'TAX': -0.06159390107057439, 'RAD': -0.0019989462268335386, 'CHAS': -0.003072638974510213, 'INDUS': -0.08894484355531622, 'B': -0.0028642973910055687, 'ZN': -0.0029794692873468575}
Prediction fidelity between original and adversarial model: 1.0
==========